In [48]:
tracker_df = pd.read_csv("budget_tracker.csv")
# Creating DataFrame
tracker_df = pd.DataFrame(tracker_df)

# Convert the 'Date' column to datetime format
tracker_df['Date'] = pd.to_datetime(tracker_df['Date'], format='%d/%m/%y')

# Create the 'MonthYear' column
tracker_df['MonthYear'] = tracker_df['Date'].dt.strftime('%m/%y')

# Drop the 'Month', 'Year', and 'Day' columns
tracker_df = tracker_df.drop(columns=['Month', 'Year', 'Day'])

# Display the updated DataFrame
tracker_df

,Date,Description,Category,Expenditure,MonthYear
0,2023-10-01,Rent payment,Home,15119.00,10/23
1,2023-10-01,Buy clothes,Other,1659.00,10/23
2,2023-10-02,"Groceries- fruits, biscuits",Food,144.09,10/23
3,2023-10-03,Travel to office and back,Auto,27.00,10/23
4,2023-10-03,Lunch,Food,38.00,10/23
...,...,...,...,...,...
542,2024-09-06,Travel to office and back,Travel,10.00,09/24
543,2024-09-06,Lunch,Food,105.00,09/24
544,2024-09-07,Lunch,Food,316.00,09/24
545,2024-09-07,Travel to airport,Entertainment,210.00,09/24


In [49]:
print(tracker_df.shape)
print(tracker_df.columns.tolist())

(547, 5)
['Date', 'Description', 'Category', 'Expenditure', 'MonthYear']


In [50]:
set_df = pd.read_csv("budget_set.csv")
print(set_df.shape)
print(set_df.columns.tolist())

(108, 3)
['MonthYear', 'Category', 'Budget']


In [52]:
import pandas as pd
from sqlalchemy import create_engine


# Create SQLite database and engine
engine = create_engine("sqlite:///budget_tracker.db")

# Add budget_tracker table
tracker_df.to_sql("budget_tracker", engine, index=False, if_exists="replace")

# Add budget_set table
set_df.to_sql("budget_set", engine, index=False, if_exists="replace")

108

In [60]:
from sqlalchemy import text

# Execute the query
query = text("""
    SELECT 
       bt.Category, 
       bt.MonthYear, 
       SUM(bt.Expenditure) as ActualSpending, 
       bs.Budget as BudgetLimit,
       (bs.Budget - SUM(bt.Expenditure)) as Difference
    FROM budget_tracker bt
    JOIN budget_set bs ON bt.Category = bs.Category AND bt.MonthYear = bs.MonthYear
    WHERE bt.MonthYear = '06/24'
    GROUP BY bt.Category, bt.MonthYear
""")
with engine.connect() as connection:
    result = connection.execute(query)
    
    # Fetch all rows and convert to a list of tuples
    rows = result.fetchall()
    print(rows)

# Convert the result to a pandas DataFrame if needed
df = pd.DataFrame(rows, columns=result.keys())

# Print the result
df

[('Auto', '06/24', 119.0, 500.0, 381.0), ('Entertainment', '06/24', 1160.0, 1500.0, 340.0), ('Food', '06/24', 4254.0, 4000.0, -254.0), ('Home', '06/24', 16207.0, 17000.0, 793.0), ('Travel', '06/24', 699.0, 1500.0, 801.0), ('Utilities', '06/24', 476.0, 1000.0, 524.0)]


,Category,MonthYear,ActualSpending,BudgetLimit,Difference
0,Auto,06/24,119.0,500.0,381.0
1,Entertainment,06/24,1160.0,1500.0,340.0
2,Food,06/24,4254.0,4000.0,-254.0
3,Home,06/24,16207.0,17000.0,793.0
4,Travel,06/24,699.0,1500.0,801.0
5,Utilities,06/24,476.0,1000.0,524.0


In [54]:
from sqlalchemy import text

# Check if there's data in the tables
check_queries = [
    "SELECT COUNT(*) FROM budget_tracker",
    "SELECT COUNT(*) FROM budget_set",
    "SELECT * FROM budget_tracker LIMIT 5",
    "SELECT * FROM budget_set LIMIT 5"
]

for query in check_queries:
    with engine.connect() as connection:
        result = connection.execute(text(query))
        print(f"Query: {query}")
        print(result.fetchall())
        print()

# Simplified query
simplified_query = text("""
    SELECT 
        bt.Category,
        SUM(bt.Expenditure) as ActualSpending,
        bs.Budget as BudgetLimit
    FROM budget_tracker bt
    JOIN budget_set bs ON bt.Category = bs.Category
    GROUP BY bt.Category
    LIMIT 5
""")

with engine.connect() as connection:
    result = connection.execute(simplified_query)
    rows = result.fetchall()
    print("Simplified Query Results:")
    print(pd.DataFrame(rows, columns=result.keys()))
    print()

# Adjusted query with proper date formatting
adjusted_query = text("""
    SELECT 
        bt.Category,
        SUM(bt.Expenditure) as ActualSpending,
        bs.Budget as BudgetLimit,
        (SUM(bt.Expenditure) - bs.Budget) as Difference
    FROM budget_tracker bt
    JOIN budget_set bs ON bt.Category = bs.Category AND 
                          strftime('%m/%Y', bt.Date) = bs.MonthYear
    GROUP BY bt.Category, bs.MonthYear
    ORDER BY Difference DESC
    LIMIT 10
""")

with engine.connect() as connection:
    result = connection.execute(adjusted_query)
    rows = result.fetchall()
    print("Adjusted Query Results:")
    print(pd.DataFrame(rows, columns=result.keys()))

Query: SELECT COUNT(*) FROM budget_tracker
[(547,)]

Query: SELECT COUNT(*) FROM budget_set
[(108,)]

Query: SELECT * FROM budget_tracker LIMIT 5
[('2023-10-01 00:00:00.000000', 'Rent payment', 'Home', 15119.0, '10/23'), ('2023-10-01 00:00:00.000000', 'Buy clothes', 'Other', 1659.0, '10/23'), ('2023-10-02 00:00:00.000000', 'Groceries- fruits, biscuits', 'Food', 144.09, '10/23'), ('2023-10-03 00:00:00.000000', 'Travel to office and back', 'Auto', 27.0, '10/23'), ('2023-10-03 00:00:00.000000', 'Lunch', 'Food', 38.0, '10/23')]

Query: SELECT * FROM budget_set LIMIT 5
[('10/23', 'Auto', 500.0), ('10/23', 'Entertainment', 1000.0), ('10/23', 'Food', 4000.0), ('10/23', 'Home', 18206.0), ('10/23', 'Medical', 0.0)]

Simplified Query Results:
        Category  ActualSpending  BudgetLimit
0           Auto        74340.00          0.0
1  Entertainment       183996.00        800.0
2           Food       395416.08       3000.0
3           Home      2440248.00      17000.0
4        Medical         10